# 청건행(16-18) 데이터 불러오기

In [ ]:
import pandas as pd 
import numpy as np
df16 = pd.read_sas('kyrbs2016.sas7bdat')
df16

In [ ]:
df17 = pd.read_sas('kyrbs2017.sas7bdat')
df17

In [ ]:
df18 = pd.read_sas('kyrbs2018.sas7bdat')
df18

# 청건행 데이터 전처리

In [ ]:
ky18 = pd.concat([df16, df17, df18])

In [ ]:
ky18 = ky18.loc[:,['HT','WT','SEX','AGE','M_SLP_HR','M_SLP_MM','M_WK_HR','M_WK_MM','M_SLP_HR_K','M_SLP_MM_K','M_WK_HR_K','M_WK_MM_K','PR_HT','M_STR','PR_BI','PA_MSC','PA_SWD_S','PA_SWD_N','PA_SWK_S','PA_SWK_N','M_SAD']]

In [ ]:
ky18.shape

In [ ]:
# 앉은시간 전처리
ky18['SitTime'] = ((ky18['PA_SWD_S'] + ky18['PA_SWD_N'])*5 + (ky18['PA_SWK_S'] + ky18['PA_SWK_N'])*2)/7
ky18['SitTime'].describe()

In [ ]:
# 앉은시간 boxplot
ky18.boxplot(column = 'SitTime')

In [ ]:
# 인덱스 초기화
ky18 = ky18.reset_index()
ky18 = ky18.iloc[:,1:]
ky18.shape

In [ ]:
# 수면시간 전처리
for i in range(len(ky18["M_SLP_HR"])):
    if ky18["M_SLP_HR"][i]<=12:
        ky18["M_SLP_HR"][i] += 24
    else:
        pass

    if ky18["M_WK_HR"][i]<=12:
        ky18["M_WK_HR"][i] +=24
    else:
        pass

for i in range(len(ky18["M_SLP_HR"])):
    ky18["SLP_WK"] = (ky18["M_WK_HR"]*60 + ky18["M_WK_MM"])-(ky18["M_SLP_HR"]*60 + ky18["M_SLP_MM"])

In [ ]:
# 평소 스테레스 인지 정도에서 값이 5인 행들 버리기
idx_nm = ky18[ky18["M_STR"] == 5].index
ky18 = ky18.drop(idx_nm)
ky18.shape

In [ ]:
# 2주 이상 연속 우울감 여부 값 바꾸기
ky18 = ky18.replace({"M_SAD":1},{"M_SAD":3})
ky18 = ky18.replace({"M_SAD":2},{"M_SAD":1})
ky18 = ky18.replace({"M_SAD":3},{"M_SAD":2})
ky18["M_SAD"].value_counts() # 1:예   2:아니오

In [ ]:
# BMI [BMI] (명목) 변수 생성
ky18['BMI'] = (ky18['WT'] / (ky18['HT'] / 100)**2)
ky18.loc[ky18['BMI'] < 25, 'BMI'] = 0
ky18.loc[ky18['BMI'] >= 25, 'BMI'] = 1
ky18['BMI'] = ky18['BMI'].astype('category')
print(ky18['BMI'].value_counts())
print("==========================")
print(ky18['BMI'])

In [ ]:
# 청건행 변수명을 국건영과 동일하게 바꾸기
df = ky18.rename(columns = {'SEX':'sex','AGE':'age','PR_HT':'D_1_1','M_STR':'BP1','PR_BI':'BO1','PA_MSC':'BE5_1','M_SAD':'BP5', 'SLP_WK':'Total_slp_wd'})

In [ ]:
# 필요한 Column만 불러오기
df1 = df.loc[:,['sex', 'age', 'D_1_1', 'BP1', 'BO1', 'BE5_1', 'BP5', 'BMI', 'Total_slp_wd', 'SitTime']]
df1.shape

# 국건영 데이터 불러오기

In [ ]:
import pandas as pd 
import numpy as np
df_a = pd.read_sas('hn16_all.sas7bdat')
df_a

In [ ]:
df_b = pd.read_sas('hn17_all.sas7bdat')
df_b

In [ ]:
df_c = pd.read_sas('hn18_all.sas7bdat')
df_c

# 국건영이랑 청건행 합친 후 최종 전처리

In [ ]:
df_all = pd.concat([df_a,df_b,df_c])

In [ ]:
df_att = df_all[["sex","age","D_1_1","Total_slp_wd","BP1","BO1","BE5_1","BE8_1","BE8_2","BP5","HE_ht","HE_wt"]]

In [ ]:
# 인덱스 초기화
df_att = df_att.reset_index()
df_att = df_att.iloc[:,1:]
df_att.shape

In [ ]:
df_att.info()

In [ ]:
# BMI [BMI] (명목) 변수 생성
df_att['BMI'] = (df_att['HE_wt'] / (df_att['HE_ht'] / 100)**2)
df_att.loc[df_att['BMI'] < 25, 'BMI'] = 0
df_att.loc[df_att['BMI'] >= 25, 'BMI'] = 1
print(df_att['BMI'].value_counts())
print("==========================")
print(df_att['BMI'])

In [ ]:
df_csn = df_att.loc[(df_att['age']>=12) & (df_att['age']<=18), :]  # 청소년 (만12세~18세) 데이터 생성
df_csn.info()

In [ ]:
# 하루 평균 앉아서 보내는 시간(분 단위) 변수 생성
df_csn["SitTime"] = df_csn["BE8_1"]*60 + df_csn['BE8_2']

In [ ]:
df_att = pd.concat([df_csn,df1])
df_att.shape

In [ ]:
df_att['BMI'] = df_att['BMI'].astype('category')

In [ ]:
df_att.loc[(df_att['BE8_1'] == 88) |(df_att['BE8_1'] == 99) , 'BE8_1'] = np.nan 
df_att.loc[(df_att['BE8_2'] == 88) |(df_att['BE8_2'] == 99) , 'BE8_2'] = np.nan 

In [ ]:
print(df_att['BE8_1'].value_counts())
print("="*14)
print(df_att['BE8_2'].value_counts())

In [ ]:
df_att = df_att.loc[:,['sex','age','D_1_1','BP1','BO1','BE5_1','BP5','BMI','Total_slp_wd','SitTime']]

In [ ]:
# 1주일간 근력운동 일수 (등간)
print(df_att['BE5_1'].value_counts()) # 8 비해당 929명, 9 모름, 무응답 : 267명 -> NA
df_att.loc[(df_att['BE5_1'] == 8) |(df_att['BE5_1'] == 9) , 'BE5_1'] = np.nan  #na 값 처리
print(df_att['BE5_1'].value_counts())
#df_att['BE5_1'].astype('int64') # na 제거하거나 대체하면 에러 안남. 

In [ ]:
# 주관적 건강상태 [D_1_1]] (순서)
from pandas.api.types import CategoricalDtype
print(df_att['D_1_1'].value_counts())
print("==========================")
df_att.loc[df_att['D_1_1'] == 9, 'D_1_1'] = np.nan
df_att['D_1_1'] = df_att['D_1_1'].astype(CategoricalDtype(ordered=True))
print(df_att['D_1_1'].value_counts())
print("==========================")
print(df_att['D_1_1'])

In [ ]:
# 2주 이상 연속 우울감 여부 [BP5] (명목)
print(df_att['BP5'].value_counts())
print("==========================")
df_att.loc[df_att['BP5'] == 8, 'BP5'] = np.nan
df_att.loc[df_att['BP5'] == 9, 'BP5'] = np.nan
df_att['BP5'] = df_att['BP5'].astype('category')
print(df_att['BP5'].value_counts())
print("==========================")
print(df_att['BP5'])

In [ ]:
# 평소 스트레스 인지정도 [BP1] (순서)
from pandas.api.types import CategoricalDtype
print(df_att['BP1'].value_counts())
print("==========================")
df_att.loc[df_att['BP1'] == 8, 'BP1'] = np.nan
df_att.loc[df_att['BP1'] == 9, 'BP1'] = np.nan

df_att['BP1'] = df_att['BP1'].astype(CategoricalDtype(ordered=True))
print(df_att['BP1'].value_counts())
print("==========================")
print(df_att['BP1'])

In [ ]:
# 주관적 체형 인지 [BO1] (순서)
from pandas.api.types import CategoricalDtype
print(df_att['BO1'].value_counts())
print("==========================")
df_att.loc[df_att['BO1'] == 8, 'BO1'] = np.nan
df_att.loc[df_att['BO1'] == 9, 'BO1'] = np.nan

df_att['BO1'] = df_att['BO1'].astype(CategoricalDtype(ordered=True))
print(df_att['BO1'].value_counts())
print("==========================")
print(df_att['BO1'])

In [ ]:
# 1주일간 근력운동 일수 [BE5_1]] (순서)
from pandas.api.types import CategoricalDtype
print(df_att['BE5_1'].value_counts())
print("==========================")
df_att['BE5_1'] = df_att['BE5_1'].astype(CategoricalDtype(ordered=True))
print(df_att['BE5_1'].value_counts())
print("==========================")
print(df_att['BE5_1'])

In [ ]:
df_att['sex'] = df_att['sex'].astype('category')

In [ ]:
df_att.info()

# 결측치 처리
### 결측치 제거 df_all
### 평균값 대체 df_mean
### 중앙값 대체 df_median
### 최빈값 대체 df_mode

In [ ]:
# 결측치 제거
df_new = df_att.dropna(axis=0)
df_new = df_new.reset_index()

from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
standard = transformer.fit_transform(df_new.loc[:,['Total_slp_wd','SitTime']])
df_standard = pd.DataFrame(standard, columns = ['Total_slp_wd_standard','SitTime_standard'])
df_standard = pd.concat([df_new, df_standard], axis = 1)
df_standard

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(df_new.loc[:,['Total_slp_wd','SitTime']])
df_scaled = pd.DataFrame(scaled, columns = ['Total_slp_wd_scaled','SitTime_scaled'])
df_all = pd.concat([df_standard, df_scaled], axis=1)
df_all = df_all.iloc[:,1:]
df_all

In [ ]:
# 평균값 대체 df
df_csn_mean = df_att.copy()

print(df_csn_mean.isnull().sum())
df_csn_mean.fillna({'D_1_1':int(df_csn_mean['D_1_1'].mode()),
                    'BP1':int(df_csn_mean['BP1'].mode()),
                    'BO1':int(df_csn_mean['BO1'].mode()),
                    'BP5':int(df_csn_mean['BP5'].mode()),
                    'BMI':int(df_csn_mean['BMI'].mode()),
                    'BE5_1':int(df_csn_mean['BE5_1'].mode()),
                    'age':int(df_csn_mean['age'].mean()),
                    'Total_slp_wd':df_csn_mean['Total_slp_wd'].mean(),
                    'SitTime':df_csn_mean['SitTime'].mean()}, inplace=True)
print('========================')
print(df_csn_mean.isnull().sum())

# 평균값 대체 자료에 표준화 및 정규화

## 인덱스 초기화
df_csn_mean.reset_index(inplace=True)
df_csn_mean = df_csn_mean.iloc[:,1:]

## 정규화
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
standard = transformer.fit_transform(df_csn_mean.loc[:,['Total_slp_wd','SitTime']])
df_standard = pd.DataFrame(standard, columns = ['Total_slp_wd_standard','SitTime_standard'])

## Min-Max Scaling
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(df_csn_mean.loc[:,['Total_slp_wd','SitTime']])
df_scaled = pd.DataFrame(scaled, columns = ['Total_slp_wd_scaled','SitTime_scaled'])

## 합치기
df_mean = pd.concat([df_csn_mean, df_standard, df_scaled], axis=1)
df_mean.head()

In [ ]:
# 중앙값 대체 df [df_median]

df_csn_median = df_att.copy()

print(df_csn_median.isnull().sum())
df_csn_median.fillna({'D_1_1':int(df_csn_median['D_1_1'].mode()),
                    'BP1':int(df_csn_median['BP1'].mode()),
                    'BO1':int(df_csn_median['BO1'].mode()),
                    'BP5':int(df_csn_median['BP5'].mode()),
                    'BMI':int(df_csn_median['BMI'].mode()),
                    'BE5_1':int(df_csn_median['BE5_1'].mode()),
                    'age':int(df_csn_mean['age'].median()),
                    'Total_slp_wd':df_csn_median['Total_slp_wd'].median(),
                    'SitTime':df_csn_median['SitTime'].median()}, inplace=True)
print('========================')
print(df_csn_median.isnull().sum())

## 인덱스 초기화
df_csn_median.reset_index(inplace=True)
df_csn_median = df_csn_median.iloc[:,1:]

## 정규화
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
standard = transformer.fit_transform(df_csn_median.loc[:,['Total_slp_wd','SitTime']])
df_standard = pd.DataFrame(standard, columns = ['Total_slp_wd_standard','SitTime_standard'])

## Min-Max Scaling
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(df_csn_median.loc[:,['Total_slp_wd','SitTime']])
df_scaled = pd.DataFrame(scaled, columns = ['Total_slp_wd_scaled','SitTime_scaled'])

## 합치기
df_median = pd.concat([df_csn_median, df_standard, df_scaled], axis=1)
df_median.head()

In [ ]:
# 최빈값 대체 df [df_mode]
df_csn_mode = df_att.copy()

print(df_csn_mode.isnull().sum())
df_csn_mode.fillna({'D_1_1':int(df_csn_mode['D_1_1'].mode()),
                    'BP1':int(df_csn_mode['BP1'].mode()),
                    'BO1':int(df_csn_mode['BO1'].mode()),
                    'BP5':int(df_csn_mode['BP5'].mode()),
                    'BMI':int(df_csn_mode['BMI'].mode()),
                    'BE5_1':int(df_csn_mode['BE5_1'].mode()),
                    'age':int(df_csn_mean['age'].mode()),
                    'Total_slp_wd':float(df_csn_mode['Total_slp_wd'].mode()),
                    'SitTime':float(df_csn_mode['SitTime'].mode())}, inplace=True)
print('========================')
print(df_csn_mode.isnull().sum())

## 인덱스 초기화
df_csn_mode.reset_index(inplace=True)
df_csn_mode = df_csn_mode.iloc[:,1:]

## 정규화
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
standard = transformer.fit_transform(df_csn_mode.loc[:,['Total_slp_wd','SitTime']])
df_standard = pd.DataFrame(standard, columns = ['Total_slp_wd_standard','SitTime_standard'])

## Min-Max Scaling
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(df_csn_mode.loc[:,['Total_slp_wd','SitTime']])
df_scaled = pd.DataFrame(scaled, columns = ['Total_slp_wd_scaled','SitTime_scaled'])

## 합치기
df_mode = pd.concat([df_csn_mode, df_standard, df_scaled], axis=1)
df_mode.head()

In [ ]:
#남자 
df_all_b = df_all.loc[df_all["sex"]==1,:]
df_all_b = df_all_b.drop('sex', axis = 1)

df_mean_b = df_mean.loc[df_mean["sex"]==1,:]
df_mean_b = df_mean_b.drop('sex', axis = 1)

df_median_b = df_median.loc[df_median["sex"]==1,:]
df_median_b = df_median_b.drop('sex', axis = 1)

df_mode_b = df_mode.loc[df_mode["sex"]==1,:]
df_mode_b = df_mode_b.drop('sex', axis = 1)

df_all_b.shape, df_mean_b.shape, df_median_b.shape, df_mode_b.shape

In [ ]:
#DU자 
df_all_g = df_all.loc[df_all["sex"]==2,:]
df_all_g = df_all_g.drop('sex', axis = 1)

df_mean_g = df_mean.loc[df_mean["sex"]==2,:]
df_mean_g = df_mean_g.drop('sex', axis = 1)

df_median_g = df_median.loc[df_median["sex"]==2,:]
df_median_g = df_median_g.drop('sex', axis = 1)

df_mode_g = df_mode.loc[df_mode["sex"]==2,:]
df_mode_g = df_mode_g.drop('sex', axis = 1)

df_all_g.shape, df_mean_g.shape, df_median_g.shape, df_mode_g.shape

## 모델링

### X에 표준화 및 정규화 또는 원형 머쓸지 골라서 drop하기

In [ ]:
X = df_mean_b.drop(['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd_standard','SitTime_standard','BMI'],axis='columns') 
y = df_mean_b['BMI']

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## 예시: RandomForest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
 
#최적 파라미터 값 찾기
param_grid = {'C':[0.001, 0.01, 0.1,100], 'gamma':[0.001, 0.01, 0.1]}

# 모델
grid_search = GridSearchCV(SVC(),param_grid, cv=3, return_train_score = True)
grid_search.fit(X_train, y_train)
predicted = grid_cv.predict(X_test)
print(confusion_matrix(y_test, predicted))
print(model_evaluation(y_test, predicted))

predicted = grid_search.predict(X_test)
print(confusion_matrix(y_test, predicted))
print(model_evaluation(y_test, predicted))

print("best parameters : {}".format(grid_search.best_params_))
#print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV

#rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
#rf.fit(X_train, y_train)

#최적 파라미터 값 찾기
params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, n_jobs = -1)
grid_cv.fit(X_train, y_train)

predicted = grid_cv.predict(X_test)
print(confusion_matrix(y_test, predicted))
print(model_evaluation(y_test, predicted))

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
rf_clf1 = RandomForestClassifier(max_depth = 10,
                                min_samples_leaf = 12,
                                min_samples_split = 8,
                                n_estimators = 100,
                                random_state = 0,
                                n_jobs = -1)
rf_clf1.fit(X_train, y_train)
predicted = rf_clf1.predict(X_test)
print(confusion_matrix(y_test, predicted))
print(model_evaluation(y_test, predicted))

# 간소화

In [ ]:
# 간소화(KNN // 여 // CV=5)
from sklearn.metrics import confusion_matrix

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    
    print("Accuracy: ", round(Accuracy,3))
    print("Precision: ", round(Precision,3))
    print("Recall: ", round(Recall,3))
    print("Specificity: ", round(Specificity,3))
    print("F1-Score: ", round(F1_Score,3))

count = 0

for df in [df_all_g, df_mean_g, df_median_g, df_mode_g]:
    for column in [['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd_standard','SitTime_standard','BMI'],
                                                    ['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd','SitTime','BMI'],
                                                    ['Total_slp_wd_standard','SitTime_standard','Total_slp_wd','SitTime','BMI']]:
        
        count = count+1
        
        if count == 1 or count  == 2 or count == 3:
            print('제목: df_all_g')
        elif count == 4 or count  == 5 or count == 6:
            print('제목: df_mean_g')
        elif count == 7 or count  == 8 or count == 9:
            print('제목: df_median_g')
        elif count == 10 or count  == 11 or count == 12:
            print('제목: df_mode_g')
        
        if count == 1 or count  == 4 or count == 7 or count == 10 :
            print('데이터: 원본')
        elif count == 2 or count  == 5 or count == 8 or count == 11:
            print('제목: Standard')
        elif count == 3 or count  == 6 or count == 9 or count == 12:
            print('제목: Scaled')
        
        X = df.drop(column,axis=1) 
        y = df['BMI']

        from sklearn.model_selection import train_test_split
        from sklearn.metrics import accuracy_score

        # test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)

        from sklearn.ensemble import RandomForestClassifier
        from sklearn.model_selection import KFold, GridSearchCV

        #rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
        #rf.fit(X_train, y_train)

        #최적 파라미터 값 찾기

        from sklearn.neighbors import KNeighborsClassifier
        knn_classifier = KNeighborsClassifier()        
        knn_parameters = [{
            'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
            'leaf_size': [5, 10, 15, 20, 25, 30, 35, 40],
            'weights': ['uniform', 'distance']}]
        # Stratified k-fold (default for classifier)
        # n = 5 folds is default
        grid_cv = GridSearchCV(estimator = knn_classifier, param_grid = knn_parameters, cv=5, scoring='recall')
        grid_cv.fit(X_train, y_train)

        predicted = grid_cv.predict(X_test)
        print(confusion_matrix(y_test, predicted))
        print(model_evaluation(y_test, predicted))


        print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
        #print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
        print('='*40)
        print()

In [ ]:
# 간소화 (RF // 남 // CV=5)
from sklearn.metrics import confusion_matrix

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    
    print("Accuracy: ", round(Accuracy,3))
    print("Precision: ", round(Precision,3))
    print("Recall: ", round(Recall,3))
    print("Specificity: ", round(Specificity,3))
    print("F1-Score: ", round(F1_Score,3))

count = 0

for df in [df_all_b, df_mean_b, df_median_b, df_mode_b]:
    for column in [['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd_standard','SitTime_standard','BMI'],
                                                    ['Total_slp_wd_scaled','SitTime_scaled','Total_slp_wd','SitTime','BMI'],
                                                    ['Total_slp_wd_standard','SitTime_standard','Total_slp_wd','SitTime','BMI']]:
        
        count = count+1
        
        if count == 1 or count  == 2 or count == 3:
            print('제목: df_all_b')
        elif count == 4 or count  == 5 or count == 6:
            print('제목: df_mean_b')
        elif count == 7 or count  == 8 or count == 9:
            print('제목: df_median_b')
        elif count == 10 or count  == 11 or count == 12:
            print('제목: df_mode_b')
        
        if count == 1 or count  == 4 or count == 7 or count == 10 :
            print('데이터: 원본')
        elif count == 2 or count  == 5 or count == 8 or count == 11:
            print('제목: Standard')
        elif count == 3 or count  == 6 or count == 9 or count == 12:
            print('제목: Scaled')
        
        X = df.drop(column,axis=1) 
        y = df['BMI']

        from sklearn.model_selection import train_test_split
        from sklearn.metrics import accuracy_score

        # test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=3)

        from sklearn.ensemble import RandomForestClassifier
        from sklearn.model_selection import KFold, GridSearchCV

        #rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
        #rf.fit(X_train, y_train)

        #최적 파라미터 값 찾기
        params = { 'n_estimators' : [10, 100],
                   'max_depth' : [6, 8, 10, 12],
                   'min_samples_leaf' : [8, 12, 18],
                   'min_samples_split' : [8, 16, 20]
                    }
        rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
        grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, n_jobs = -1)
        grid_cv.fit(X_train, y_train)

        predicted = grid_cv.predict(X_test)
        print(confusion_matrix(y_test, predicted))
        print(model_evaluation(y_test, predicted))

        print('하이퍼 파라미터: ', grid_cv.best_params_)
        #print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
        print('='*40)
        print()